You can access the accompanying video here: 

<a href="https://blinks.bloomberg.com/screens/PLYR%20VOD%20362284390"> Eco Data and Animations (10 min)</a>

In [1]:
# This is how we can see where our project files live on our local drive
x = %pwd

print(x)

C:\Users\traveler\AppData\Local\bipy\6316781\projects\209074518baf4c01ac3b169e65c5d60c


In [2]:
# Import our relevant libraries
import bql 
import pandas as pd 
# Start the BQL Service
bq = bql.Service()

In [3]:
# After we import our csv into our project files, we can use pd.read_csv
# This will turn our csv into a pandas dataframe
file_read = pd.read_csv('state_tickers.csv')

In [4]:
# We will use the claims tickers in the csv as our claims universe
# We will use the labor tickers in the csv as our labor universe
claims_tickers = bq.univ.list(list(file_read['Claims Tickers']))
labor_tickers = bq.univ.list(list(file_read['Labor Force Tickers']))

In [5]:
# We can use the window function to cut our unemployment data into chunks of 4
# We will take the sum of each chunk
unemploy_field = bq.func.sum(
    bq.func.window(bq.data.px_last(dates=bq.func.range('2020-05-31 -1y+1w','2020-05-31')).dropna(),4,windowstep=4))

In [6]:
# Define our unemployment request
unreq = bql.Request(claims_tickers,{'unemployment claims':unemploy_field})
# Execute our unemployment request
unresp = bq.execute(unreq)

In [7]:
# Put the response into a dataframe and reset the index
undf = unresp[0].df().reset_index()

In [8]:
# Define our field for our labor tickers
lab_field = bq.data.px_last(dates=bq.func.range('-1y','0d')).dropna()

In [9]:
# Define our labor request
lab_req = bql.Request(labor_tickers,{'Labor force':lab_field})
# Execute our labor request
lab_resp = bq.execute(lab_req)

In [10]:
# Put the response into a dataframe and reset the index
labdf = lab_resp[0].df().reset_index()

In [11]:
# Create our dictionaries for mapping our tickers to our regions
claims_dict = dict(zip(file_read['Claims Tickers'],file_read['Region']))
labor_dict = dict(zip(file_read['Labor Force Tickers'],file_read['Region']))

In [12]:
# Map the dictionaries to the Region column
undf['Region'] = undf['ORIG_IDS'].map(claims_dict)

In [13]:
# Map the dictionaries to the region column
labdf['Region'] = labdf['ID'].map(labor_dict)

In [14]:
# concatenate the dataframes
merged = pd.concat([labdf,undf['unemployment claims']],axis=1)

In [15]:
# Adjust the data for the labor force
merged['claims/labor'] = merged['unemployment claims']/merged['Labor force'] *100

In [16]:
# specify which columns you're looking for 
merged_final = merged[['DATE','Region','claims/labor']]

In [17]:
# Groupby region and date 
df_for_vis = merged_final.groupby(['Region','DATE']).mean().reset_index()

In [18]:
# Create the visualization
import pandas as pd
import numpy as np
from bqplot import LinearScale, OrdinalScale, Label, Axis, Figure, Bars
import ipywidgets as widgets

# Define column names for dataframe
VALUE_COLUMN = 'claims/labor'
DATE_COLUMN = 'DATE'
CATEGORY_COLUMN = 'Region'

# Define visualization labels
CHART_TITLE = 'Horizontal bar chart with animation'
X_AXIS_LABEL = 'Last Price'



df = df_for_vis

# Set initial data as first date of time range
dates_arr = df[DATE_COLUMN].unique()
first_entry = df[df[DATE_COLUMN] == dates_arr[0]]

# Create scales
scale_x = OrdinalScale(allow_padding=True)

# Set the maximum range of the domain to allow for readability of label marks
scale_y = LinearScale(max=df[VALUE_COLUMN].max()*1.15)

# Create label mark
label = Label(x=first_entry[VALUE_COLUMN],
              y=first_entry[CATEGORY_COLUMN],
              text=[round(elem, 2) for elem in first_entry[VALUE_COLUMN]],
              scales={'y': scale_x, 'x': scale_y},
              default_size=12,
              colors=['white'],
              x_offset=12)

# Create bar mark
bar = Bars(x=first_entry[CATEGORY_COLUMN],
           y=first_entry[VALUE_COLUMN],
           scales={'x': scale_x, 'y': scale_y},
           orientation='horizontal',
           color_mode='auto',
           colors=['#000000', '#0073ff', '#fa5a28', '#c873ff',
                   '#d7be00','#00c2d7', '#db8922', '#f328bb',
                   '#50f06e', '#8f52b6','#b09b00'])

# Create axes
axis_x = Axis(scale=scale_x, orientation='vertical', tick_style={'font-size':'14px'},
            grid_lines='none')
axis_y = Axis(scale=scale_y, tick_format='0.2f', label=X_AXIS_LABEL)

# Create figure
figure = Figure(marks=[bar, label],
                    axes=[axis_x, axis_y],
                    padding_x=0,
                    padding_y=0.025,
                    fig_margin={'top': 60, 'bottom':60,
                               'left':170,'right':100},
                    title=CHART_TITLE,
                    animation_duration=500,
                    layout={'width':'auto'})

# Create slider
int_slider = widgets.IntSlider(min=0,
                               max=len(dates_arr) - 1,
                               step=1,
                               continuous_update=False,
                               readout=False)

# Create Html widget
date_selected_html = widgets.HTML(layout={'width':'50%'})

# Callback function to update graph view upon change to slider
def on_value_change(change):
    if change is not None and change['new'] is not None:
        # Selected value from the slider
        selected_num = change['new']

        # Get date associated with number on slider
        selected_date = dates_arr[selected_num]

        new_data_entry = df.loc[df[DATE_COLUMN] == selected_date]

        # Displaying this format via the html object
        selected_timestamp = new_data_entry[DATE_COLUMN].iloc[0]
        date_selected_html.value = selected_timestamp.strftime('%Y-%m-%d')

        # Determine correct ordering of sectors such that they are displayed in
        # descending order of prices
        date_df = pd.DataFrame(new_data_entry)
        date_df = date_df.sort_values(VALUE_COLUMN)

        # Set order of domain
        category_order = date_df[CATEGORY_COLUMN].tolist()
        scale_x.domain = category_order
        # Update new data for bars and labels
        bar.y = new_data_entry[VALUE_COLUMN]
        label_data = new_data_entry[VALUE_COLUMN]

        # Add padding between bars and labels
        label.text = [round(elem, 2) for elem in label_data]
        label_data = [entry for entry in label_data]
        label.x = label_data

int_slider.observe(on_value_change, names='value')

# Create play button and add to graph
play_button = widgets.Play(min=0, max=len(dates_arr) - 1, interval=500)
widgets.jslink((play_button, 'value'), (int_slider, 'value'))

# Put together all components in same display
h_box = widgets.HBox([play_button, int_slider, date_selected_html],
                     layout={'display':'flex', 'align_self':'center'})
visualization = widgets.VBox([h_box, figure])
visualization


In [19]:
f = bq.data.px_last(dates=bq.func.range('-1y','0d',FRQ='W'),fill='prev')